In [43]:
import copy
from pathlib import Path
import hyp3_sdk as sdk
import geopandas as gpd 
import asf_search as asf

In [53]:
burst = '115_245676_IW2'

project_name = burst
work_dir = Path.cwd() / project_name
stac_dir = work_dir / 'stac'
work_dir.mkdir(parents=True, exist_ok=True)

In [54]:
hyp3 = sdk.HyP3()

jobs = hyp3.find_jobs(name=project_name)


In [55]:
# Haven't processed anything under this name
jobs

Batch([])

In [56]:
project_name

'115_245676_IW2'

In [57]:
results = asf.search(
    platform=asf.PLATFORM.SENTINEL1,
    processingLevel=asf.PRODUCT_TYPE.BURST,
    polarization='VV',
    fullBurstID=project_name
)
print(len(results))


158


In [72]:
gf = gpd.GeoDataFrame.from_features(results.geojson(), crs=4326)
gf.explore(tiles='EsriWorldImagery',style_kwds=dict(fill=False))

In [73]:
print('User:{user_id} Quota:{remaining_credits}'.format(**hyp3.my_info()))

User:scottyhq Quota:2989


In [74]:
gf['datetime'] = gpd.pd.to_datetime(gf.startTime)

In [75]:
gf.sort_values('datetime', inplace=True)

In [76]:
gf.datetime.describe()

count                                    158
mean     2021-07-21 22:08:34.858660096+00:00
min         2019-01-08 14:14:24.748574+00:00
25%      2020-04-23 14:14:31.125656064+00:00
50%      2021-08-04 14:14:21.990089472+00:00
75%      2022-10-04 14:14:51.324360448+00:00
max         2024-03-06 14:14:52.290372+00:00
Name: datetime, dtype: object

In [78]:
gf.datetime.diff().describe()

count                           157
mean     12 days 00:00:00.175425465
std       2 days 23:04:15.413061903
min          5 days 23:59:17.891010
25%         11 days 23:59:59.664352
50%         12 days 00:00:00.182922
75%         12 days 00:00:00.635498
max         35 days 23:59:59.504379
Name: datetime, dtype: object

In [79]:
subset = gf.set_index('datetime')

In [80]:
subset['date'] = gpd.pd.to_datetime(subset.startTime) #.dt.strftime('%Y%m%d')

In [81]:
job_definition = {
  "name": None,
  "job_type": "INSAR_ISCE_BURST",
  "job_parameters": {
      "granules": [None,None],
      "apply_water_mask": False,
      "looks": "5x1", # '20x4', '10x2', '5x1' default=20x4
  },
}
job_definition

{'name': None,
 'job_type': 'INSAR_ISCE_BURST',
 'job_parameters': {'granules': [None, None],
  'apply_water_mask': False,
  'looks': '5x1'}}

In [82]:
# create interferogram job definitions
jobs = [] # List of job dictionaries

#for i in [0,1,2]: 
for i in range(len(subset)-1):
    jobdef = copy.deepcopy(job_definition) #Avoid overwriting in-memory!
    
    ref = subset.iloc[i]
    sec = subset.iloc[i+1]
    jobname = f"{burst}_{ref.date.strftime('%Y%m%d')}_{sec.date.strftime('%Y%m%d')}_5rlks"

    jobdef['name'] = jobname
    jobdef['job_parameters']['granules'] = [ref.fileID, sec.fileID]
    jobs.append(jobdef)

In [83]:
jobs[0]

{'name': '115_245676_IW2_20190108_20190120_5rlks',
 'job_type': 'INSAR_ISCE_BURST',
 'job_parameters': {'granules': ['S1_245676_IW2_20190108T141423_VV_D96D-BURST',
   'S1_245676_IW2_20190120T141423_VV_D8A3-BURST'],
  'apply_water_mask': False,
  'looks': '5x1'}}

In [84]:
jobs = hyp3.submit_prepared_jobs(jobs)

In [85]:
hyp3.watch(jobs)

  0%|          | 0/157 [timeout in 10800 s]

Batch([Job.from_dict({'job_id': '20da76bb-0658-499e-8140-c46dcace40f9', 'job_type': 'INSAR_ISCE_BURST', 'request_time': '2024-03-10T19:22:46+00:00', 'status_code': 'SUCCEEDED', 'user_id': 'scottyhq', 'name': '115_245676_IW2_20190108_20190120_5rlks', 'job_parameters': {'apply_water_mask': False, 'granules': ['S1_245676_IW2_20190108T141423_VV_D96D-BURST', 'S1_245676_IW2_20190120T141423_VV_D8A3-BURST'], 'looks': '5x1'}, 'files': [{'filename': 'S1_245676_IW2_20190108_20190120_VV_INT20_FEF4.zip', 's3': {'bucket': 'hyp3-edc-prod-contentbucket-1fv14ed36ifj6', 'key': '20da76bb-0658-499e-8140-c46dcace40f9/S1_245676_IW2_20190108_20190120_VV_INT20_FEF4.zip'}, 'size': 248331427, 'url': 'https://d3gm2hf49xd6jj.cloudfront.net/20da76bb-0658-499e-8140-c46dcace40f9/S1_245676_IW2_20190108_20190120_VV_INT20_FEF4.zip'}], 'logs': [], 'browse_images': ['https://d3gm2hf49xd6jj.cloudfront.net/20da76bb-0658-499e-8140-c46dcace40f9/S1_245676_IW2_20190108_20190120_VV_INT20_FEF4_unw_phase.png'], 'thumbnail_images'

In [90]:
# Waits and updates final job status
jobs = hyp3.watch(jobs)

  0%|          | 0/157 [timeout in 10800 s]

In [91]:
# requires https://github.com/ASFHyP3/hyp3-sdk/pull/266
from hyp3_sdk.stac import stac

stac.create_stac_collection(jobs, stac_dir)

100%|██████████| 157/157 [00:11<00:00, 13.63it/s]


In [95]:
import pystac
from hyp3_sdk.stac import load


collection = pystac.Collection.from_file(stac_dir / 'collection.json')
items = list(collection.get_all_items())
dataset = load.create_xarray_dataset(items)

In [96]:
dataset

<xarray.Dataset> Size: 55GB
Dimensions:        (time: 157, y: 2028, x: 5446)
Coordinates:
  * y              (y) float64 16kB 4.91e+06 4.91e+06 ... 4.869e+06 4.869e+06
  * x              (x) float64 44kB 5.467e+05 5.467e+05 ... 6.556e+05 6.556e+05
    spatial_ref    int32 4B 32610
  * time           (time) datetime64[ns] 1kB 2019-01-14T14:14:23 ... 2024-02-...
Data variables:
    corr           (time, y, x) float32 7GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    unw_phase      (time, y, x) float32 7GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    lv_phi         (time, y, x) float32 7GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    lv_theta       (time, y, x) float32 7GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    dem            (time, y, x) float32 7GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    water_mask     (time, y, x) float32 7GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    conncomp       (time, y, x) float32 7GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    wrapped_phase  (time, y, x) float32 7GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>